# Python 2019/2020Z: Term Project - Polat, Su
## HR Management App
### Required Imports

### Creating Database Tables

In [ ]:
import sqlite3
import time
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response


conn = sqlite3.connect('database.db')
c = conn.cursor()
isLoggedIn = 0
loginResult = None
topSalariesResult = None

# Create table - Personnel
query='''CREATE TABLE IF NOT EXISTS Personnel (
                        pID INTEGER PRIMARY KEY,
                        name TEXT NOT NULL,
                        surname TEXT NOT NULL,
                        email text NOT NULL UNIQUE,
                        zip_code INTEGER,
                        salary INTEGER, 
                        job_start_date datetime,
                        password char(20) NOT NULL,
                        personnel_type char(1) NOT NULL,
                        rank INTEGER);'''

c.execute(query)

# Create table - AdministrativePersonnel
query='''CREATE TABLE IF NOT EXISTS AdministrativePersonnel (
                        apID INTEGER PRIMARY KEY,
                        private_health_insurance INTEGER,
                        expense INTEGER);'''
c.execute(query)


# Create table - FieldPersonnel
query='''CREATE TABLE IF NOT EXISTS FieldPersonnel (
                        fpID INTEGER PRIMARY KEY,
                        shift_type INTEGER,
                        yearly_bonus INTEGER);'''
c.execute(query)


# Create table - ExecutivePersonnel
query='''CREATE TABLE IF NOT EXISTS ExecutivePersonnel (
                        epID INTEGER PRIMARY KEY,
                        premium_ratio INTEGER);'''
c.execute(query)


# Create table - Address
query='''CREATE TABLE IF NOT EXISTS Address (
                        zip_code_address char(5) PRIMARY KEY,
                        street char(40),
                        building char(40),
                        city char(40));'''
c.execute(query)



#Fill tables with data
#Add an admin account

query = '''INSERT INTO Personnel(pID,name,surname,email,password,personnel_type) 
            SELECT 000, 'Global','Admin','admin','admin','e' 
            WHERE NOT EXISTS(SELECT 1 FROM Personnel WHERE pID = 000)'''

c.execute(query)


#Fill Address Table
"""
query = '''INSERT INTO Address(zip_code_address,street,building,city) 
            SELECT 31313 as zip_code_address,'Hinesville' as street,'82' as building,'GA' as city 
            UNION select 96815,'Honolulu','13','HI'
            UNION select 21801,'Salisbury','21','MD'
            UNION select 42141,'Glasgow','55','KY'
            UNION select 33308,'Fort Lauderdale','C5/D','FL'
            UNION select 48127,'Dearborn Heights','28','MI'
            UNION select 29710,'Clover','13','SC'
            UNION select 17042,'Lebanon','43','PA'
            UNION select 30101,'Acworth','17','GA'
            UNION select 18064,'Nazareth','865','PA'
            UNION select 54935,'Fond Du Lac','13','WI'
            UNION select 11412,'Saint Albans','43','NY'
            UNION select 02904,'Providence','665','RI'
            UNION select 07410,'Fair Lawn','89','NJ'
            UNION select 44203,'Barberton','456','OH'
            UNION select 71730,'El Dorado','34','AR'
            UNION select 77566,'Lake Jackson','55','TX'
            UNION select 29406,'Charleston','63','SC'
            UNION select 48640,'Midland','76','MI'
            UNION select 10952,'Monsey','345','NY' '''
            
c.execute(query)
"""

#Get Zip Codes
query = '''select zip_code_address from Address'''

c.execute(query)
zipCodes = c.fetchall()





conn.commit()




def create_connection(db_file="database.db"):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
 
    return conn

def select_login_info(conn, email, password):
    cur = conn.cursor()
    cur.execute("SELECT * FROM Personnel where email = ? and password= ?", (email,password,))
    rows = cur.fetchall()
    conn.close()
    return rows






@route('/<filename:path>')
def send_static(filename):
    return static_file(filename, root='static/')

@route('/', methods=['GET', 'POST'])
@route('/index')
def index(name=''):
    return template('index',msg='')


@route('/login')
@route('/login/')
@route('/login', method='POST')
def login():
    email = request.forms.get('email', default=False)
    password = request.forms.get('password', default=False)
    global loginResult
    loginResult = select_login_info(create_connection(),email,password)
    if (len(loginResult) > 0):
        global isLoggedIn
        isLoggedIn = 1
        if(loginResult[0][8]=="e"):
            #Getting top salaries by grouping SQL
            conn = create_connection()
            cur = conn.cursor()
            query = '''select name,surname, personnel_type, max(salary) as maxSalary from Personnel group by personnel_type'''
            cur.execute(query)
            global topSalariesResult
            topSalariesResult=cur.fetchall()
            conn.close()
            return template('home',loggedUser=loginResult, msg='You have the admin priviledges.',topSalaries=topSalariesResult)
        else:
            return template('home',loggedUser=loginResult, msg='')
    else:
        return template('index',msg="Wrong credentials.")

    
@route('/logout')
@route('/logout/')
def logout(name=''):
    global isLoggedIn
    isLoggedIn = 0
    global loginResult
    loginResult = None
    return template('index',msg="Logout successful.")
    
    
@route('/home')
@route('/home/')
def home(name=''):
    if (isLoggedIn == 1):
        if(loginResult[0][8]=="e"):
            return template('home',loggedUser=loginResult, msg='You have the admin priviledges.',topSalaries=topSalariesResult)
        else:
            return template('home',loggedUser=loginResult, msg='')
    else:
        return template('index',msg="You must log in.")
    
    
@route('/profile')
@route('/profile/')
def profile(name=''):
    if (isLoggedIn == 1):
        return template('profile',account=loginResult[0])
    else:
        return template('index',msg="You must log in.")
    
    
@route('/addPersonnel')
@route('/addPersonnel/', method='GET')
def addpersonnel(name=''):
    if (isLoggedIn == 1):
        return template('addPersonnel', zipList=zipCodes, msg="")
    else:
        return template('index',msg="You must log in.")

    
@route('/addPersonnelPost')
@route('/addPersonnelPost/')
@route('/addPersonnelPost', method='POST')
def addpersonnelpost():
    #Establishing a new connection
    conn = create_connection()
    cur = conn.cursor()
    query = '''select count(*) from Personnel'''
    cur.execute(query)
    pid=cur.fetchall()[0][0]
    
    #Getting data from Add Personnel TPL
    name = request.forms.get('name', default=False)
    surname = request.forms.get('surname', default=False)
    email = request.forms.get('email', default=False)
    zipCode = request.forms.get('zipCode', default=False)
    salary = request.forms.get('salary', default=False)
    jobStartDate = request.forms.get('jobStartDate', default=False)
    password = request.forms.get('password', default=False)
    personnelType = request.forms.get('personnelType', default=False)
    rank = request.forms.get('rank', default=False)
    
    query = '''select count(*) from Personnel where email=?'''
    cur.execute(query,(email,))
    selectEmail=cur.fetchall()[0][0]
    
    if(selectEmail==1):
        return template('addPersonnel', zipList=zipCodes, msg="A record exists under this e-mail")
    
    if personnelType in ("a","e","f"):
        #If personnel type is selected which is a required field on HTML then start instering values
        query = '''INSERT INTO Personnel
                    VALUES (?,?,?,?,?,?,?,?,?,?)'''
        cur.execute(query,(pid,name,surname,email,zipCode,salary,jobStartDate,password,personnelType,rank))
    
        # Create table for this specific personnel- RankSalaryID
        rankSalaryTableName="RankSalary"+str(pid)
        query='''CREATE TABLE IF NOT EXISTS {0} (
                                rank INTEGER,
                                salary INTEGER);'''.format(rankSalaryTableName)
        cur.execute(query)
        
        # Add values to that table
        query = '''INSERT INTO {0}
                    VALUES (?,?)'''.format(rankSalaryTableName)
        cur.execute(query,(rank,salary))
        
        #If the personnel type is administrative then expense and private healt insurance information should be added
        if(personnelType=="a"):
            expense = request.forms.get('expense', default=False)
            privateHealthInsurance = request.forms.get('privateHealthInsurance', default=False)
            
            query = '''INSERT INTO AdministrativePersonnel
                        VALUES (?,?,?)'''
            cur.execute(query,(pid,privateHealthInsurance,expense))
            conn.commit()
            conn.close()
            return template('home',loggedUser=loginResult, msg="Personnel added successfully.") 
        
        #If the personnel type is executive then premium ratio information should be added
        if(personnelType=="e"):
            premiumRatio = request.forms.get('premiumRatio', default=False)

            query = '''INSERT INTO ExecutivePersonnel
                        VALUES (?,?)'''
            cur.execute(query,(pid,premiumRatio))
            conn.commit()
            conn.close()
            return template('home',loggedUser=loginResult, msg="Personnel added successfully.")
        
        #If the personnel type is field then yearly bonus and shift type information should be added
        if(personnelType=="f"):
            yearlyBonus = request.forms.get('yearlyBonus', default=False)
            shiftType = request.forms.get('shiftType', default=False)

            query = '''INSERT INTO FieldPersonnel
                        VALUES (?,?,?)'''
            cur.execute(query,(pid,shiftType,yearlyBonus))
            conn.commit()
            conn.close()
            return template('home',loggedUser=loginResult, msg="Personnel added successfully.")
        
        
    else:
        conn.commit()
        conn.close()
        return template('index',msg="An error happened on adding new personnel.")


@route('/displayPersonnel')
@route('/displayPersonnel/')
def addpersonnel():
    if (isLoggedIn == 1):
        return template('displayPersonnel', msg="")
    else:
        return template('index',msg="You must log in.")   

@route('/selectedPersonnel')
@route('/selectedPersonnel/')
@route('/selectedPersonnel', method='POST')
def addpersonnelpost():
    #Establishing a new connection
    conn = create_connection()
    cur = conn.cursor()
    email = request.forms.get('email', default=False)
    query = '''select * from Personnel where email=?'''
    cur.execute(query,(email,))
    selectedPersonnel=cur.fetchall()
    if(len(selectedPersonnel)==1):
        #if selected personnel is an executive then it can only be displayed by another executive
        if (selectedPersonnel[0][8]=="e") and (loginResult[0][8]!="e"):
            return template('displayPersonnel', msg="You do not have prividleges to display an Executive's information.")
        else:
            return template('selectedPersonnel', msg="Success! Now route this page to selectedPersonnel")
    else:
        return template('displayPersonnel', msg="Personnel not found under this e-mail")


run(host='localhost', port=8049)








Bottle v0.11.6 server starting up (using WSGIRefServer())...
Listening on http://localhost:8049/
Hit Ctrl-C to quit.

127.0.0.1 - - [05/Jan/2020 12:23:38] "GET / HTTP/1.1" 200 747
127.0.0.1 - - [05/Jan/2020 12:23:41] "POST /login HTTP/1.1" 200 996
127.0.0.1 - - [05/Jan/2020 12:23:42] "GET /addPersonnel HTTP/1.1" 200 6348
127.0.0.1 - - [05/Jan/2020 12:24:03] "POST /addPersonnelPost HTTP/1.1" 200 1025
127.0.0.1 - - [05/Jan/2020 12:24:15] "GET /logout HTTP/1.1" 200 765
127.0.0.1 - - [05/Jan/2020 12:24:17] "POST /login HTTP/1.1" 200 1558
127.0.0.1 - - [05/Jan/2020 12:24:32] "GET /logout HTTP/1.1" 200 765
127.0.0.1 - - [05/Jan/2020 12:24:35] "POST /login HTTP/1.1" 200 996


### Create Table Objects With Queries

In [ ]:
class Personnel_Library():
    def __init__(self):
        self.create_connection()
    def create_connection(self):
        self.connection=sqlite3.connect('database.db')
        self.cursor=self.connection.cursor()
        query='''CREATE TABLE IF NOT EXISTS Personnel (
                                pID INTEGER PRIMARY KEY,
                                name TEXT NOT NULL,
                                surname TEXT NOT NULL,
                                zip_code INTEGER,
                                pay_grade INTEGER,
                                email text NOT NULL UNIQUE, 
                                job_start_date datetime,
                                password char(20) NOT NULL);'''
        self.cursor.execute(query)
        self.connection.commit()
        self.connection.close()
    def stop_connection(self):
        self.connection.close()
    def show_Personnel(self):
        query='Select * From Firm_Data_Person'
        self.cursor.execute(query)
        Firm_Data_Person=self.cursor.fetchall()
        if (len(Firm_Data_Person)==0):
            print('No Workers Here')
        else:
            for i in Firm_Data_Person:
                personnel=Personnel(i[0],i[1],i[2],i[3],i[4])
                print(personnel)
    def workers_test(self,name):
        query='Select * Firm_Data_Person where name = ?'
        self.cursor.execute(query,(name,))
        Firm_Data_Person=self.cursor.fetchall()
        if(len(Firm_Data_Person)==0):
            print('No here')
        else:
            personnel=Personnel(Firm_Data_Person[0][0],Firm_Data_Person[0][1],Firm_Data_Person[0][2],Firm_Data_Person[0][3],Firm_Data_Person[0][4])
            print(personnel)
    def add_workers(self,Personnel):
        query = "Insert into Firm_Data_Person Values(?,?,?,?,?)"

        self.cursor.execute(query,(Personnel.name,Personnel.surname,Personnel.zip_code,Personnel.pay_grade,Personnel.email,Personnel.jobstart_time))

        self.connection.commit()
    def delete_workers(self,name):

        query = "Delete From Firm_Data_Person where name = ?"

        self.cursor.execute(query,(name,))

        self.connection.commit()
    def increase_pressure(self,name):

        query = "Select * From Firm_Data_Person where name = ?"

        self.cursor.execute(query,(name,))


        Firm_Data_Person = self.cursor.fetchall()

        if (len(Firm_Data_Person) == 0):
            print("NO HERE...")
        else:
            pressure = labrar1[0][4]

            pressure += 1

            query2 = "Update Firm_Data_Person set pressure = ? where name = ?"

            self.cursor.execute(query2,(pressure,name))

            self.connection.commit()
        


In [ ]:

class Personnel_Library():
    def __init__(self):
        self.create_connection()
    def create_connection(self):
        self.connection=sqlite3.connect('FirmData8.db')
        self.cursor=self.connection.cursor()
        query='''CREATE TABLE Firm_Data_Person (
                                id INTEGER PRIMARY KEY,
                                name TEXT NOT NULL,
                                surname TEXT NOT NULL,
                                zip_code INTEGER,
                                pay_grade INTEGER,
                                email text NOT NULL UNIQUE, 
                                Jobstart_time datetime);'''
        self.cursor.execute(query)
        self.connection.commit()
    def stop_connection(self):
        self.connection.close()
    def show_Personnel(self):
        query='Select * From Firm_Data_Person'
        self.cursor.execute(query)
        Firm_Data_Person=self.cursor.fetchall()
        if (len(Firm_Data_Person)==0):
            print('No Workers Here')
        else:
            for i in Firm_Data_Person:
                personnel=Personnel(i[0],i[1],i[2],i[3],i[4])
                print(personnel)
    def workers_test(self,name):
        query='Select * Firm_Data_Person where name = ?'
        self.cursor.execute(query,(name,))
        Firm_Data_Person=self.cursor.fetchall()
        if(len(Firm_Data_Person)==0):
            print('No here')
        else:
            personnel=Personnel(Firm_Data_Person[0][0],Firm_Data_Person[0][1],Firm_Data_Person[0][2],Firm_Data_Person[0][3],Firm_Data_Person[0][4])
            print(personnel)
    def add_workers(self,Personnel):
        query = "Insert into Firm_Data_Person Values(?,?,?,?,?)"

        self.cursor.execute(query,(Personnel.name,Personnel.surname,Personnel.zip_code,Personnel.pay_grade,Personnel.email,Personnel.jobstart_time))

        self.connection.commit()
    def delete_workers(self,name):

        query = "Delete From Firm_Data_Person where name = ?"

        self.cursor.execute(query,(name,))

        self.connection.commit()
    def increase_pressure(self,name):

        query = "Select * From Firm_Data_Person where name = ?"

        self.cursor.execute(query,(name,))


        Firm_Data_Person = self.cursor.fetchall()

        if (len(Firm_Data_Person) == 0):
            print("NO HERE...")
        else:
            pressure = labrar1[0][4]

            pressure += 1

            query2 = "Update Firm_Data_Person set pressure = ? where name = ?"

            self.cursor.execute(query2,(pressure,name))

            self.connection.commit()
        
        


class Field_Personnel(Personnel):
        def __init__(self,name,surname,email,pay_grade,zip_code,jobstart_date,yearly_bonus,shift_type):
            print('Class of Field_Personnel init function')
            self.name=name
            self.surname=surname
            self.jobstart_date=jobstart_date
            self.pay_grade=pay_grade
            self.department=department
            self.zip_code=zip_code
            self.yearly_bonus=yearly_bonus
            self.shift_type=shift_type
        def change_name(self,new_name):
            print('Name Changed')
        self.name=new_name
        def change_surname(self,new_surname):
        print('Surname Changed')
        self.surname=new_surname
        def change_email(self,new_email):
        print('Changing Email Adress..')
        self.email=new_email
        def change_ZipCode(self,new_ZipCode):
        print('Zip Code changed')
        self.zip_code=new_ZipCode
        def change_jobstart_date(self,new_date):
        print('Date changed')
        self.jobstart_date=new_date
        def pay_grade_increase(self,accrual):
        print('Wage is increasing...')
        self.pay_grade+=accrual
        def pay_grade_decreasing(self,cutback):
        print('Wage is Decresing...')
        self.pay_grade-= cutback
        def yearly_bonus_increase(self,accrual):
        print('Yearly_Bonus is increasing...')
        self.yearly_bonus+=accrual
        def yearly_bonus_decreasing(self,cutback):
        print('Yearly_Bonus is Decresing...')
        self.yearly_bonus-= cutback
        def show_informations(self):
        print('Field_Personnel Informations Prepared...')
        def __str__(self):

            return "Name: {}\nSurname: {}\nPay_Grade: {}\nDepartment: {}\nEmail Adress: {}\nJobSStart_Date{}\nYearly_Bonus: {}\nShift_Type:{}\n".format(self.name,self.surname,self.pay_grade,self.email,self.email,self.jobstart_date,self.yearly_bonus,self.shift_type)
class library2():
    def __init__(self):
        self.create_connection()
    def create_connection(self):
        self.connection=sqlite3.connect('FirmData8.db')
        self.cursor=self.connection.cursor()
        query='CREATE TABLE IF NOT EXISTS labrar2(Name TEXT,Surname TEXT,Wage INT,Department TEXT,Age INT,Languages TEXT)'
        self.cursor.execute(query)
        self.connection.commit()
    def stop_connection(self):
        self.connection.close()
    def show_managers(self):
        query='Select * From labrar2'
        self.cursor.execute(query)
        labrar2=self.cursor.fetchall()
        if (len(labrar2)==0):
            print('No Workers Here')
        else:
            for i in labrar2:
                Managers=managers(i[0],i[1],i[2],i[3],i[4])
                print(Managers)
    def managers_test(self,name):
        query='Select * From labrar2 where name = ?'
        self.cursor.execute(query,(name,))
        labrar2=self.cursor.fetchall()
        if(len(labrar2)==0):
            print('No here')
        else:
            Managers=managers(labrar2[0][0],labrar2[0][1],labrar2[0][2],labrar2[0][3],labrar2[0][4])
            print(Managers)
    def add_managers(self,managers):
        query = "Insert into labrar2 Values(?,?,?,?,?)"

        self.cursor.execute(query,(managers.name,managers.surname,managers.wage,managers.department,managers.age,managers.languages))

        self.connection.commit()
    def delete_managers(self,name):

        query = "Delete From labrar2 where name = ?"

        self.cursor.execute(query,(name,))

        self.connection.commit()
    def increase_pressure(self,name):

        query = "Select * From labrar2 where name = ?"

        self.cursor.execute(query,(name,))


        labrar2 = self.cursor.fetchall()

        if (len(labrar2) == 0):
            print("NO HERE...")
        else:
            pressure = labrar2[0][4]

            pressure += 1

            query2 = "Update labrar2 set pressure = ? where name = ?"

            self.cursor.execute(query2,(pressure,name))

            self.connection.commit()


        def change_name(self,new_name):
            print('Name is Changing...')
            self.name=new_name
        def change_surname(self,new_surname):
            print('Surname is Changing...')
            self.surname=new_surname
    
class labors(workers):
    def __init__(self,name,surname,wage,department,profession):
        super().__init__(name,surname,wage,department)
        print('Class of Labors init function')
        self.profession=profession
    def show_informations(self):
        print('Labors Informations Prepared...')
        def __str__(self):

            return "Name: {}\nSurname: {}\nWage: {}\nDepartment: {}\nProfession: {}\n".format(self.name,self.surname,self.wage,self.department,self.profession)
class library3():
    def __init__(self):
        self.create_connection()
    def create_connection(self):
        self.connection=sqlite3.connect('library3.db')
        self.cursor=self.connection.cursor()
        query='CREATE TABLE IF NOT EXISTS labrar3(Name TEXT,Surname TEXT,Wage INT,Department TEXT,Profession TEXT)'
        self.cursor.execute(query)
        self.connection.commit()
    def stop_connection(self):
        self.connection.close()
    def show_labors(self):
        query='Select * From labrar3'
        self.cursor.execute(query)
        labrar3=self.cursor.fetchall()
        if (len(labrar3)==0):
            print('No Workers Here')
        else:
            for i in labrar3:
                Labors=labors(i[0],i[1],i[2],i[3],i[4])
                print(Labors)
    def labors_test(self,name):
        query='Select * From labrar3 where name = ?'
        self.cursor.execute(query,(name,))
        labrar3=self.cursor.fetchall()
        if(len(labrar3)==0):
            print('No here')
        else:
            Labors=labors(labrar3[0][0],labrar3[0][1],labrar3[0][2],labrar3[0][3],labrar3[0][4])
            print(Labors)
    def add_managers(self,managers):
        query = "Insert into labrar3 Values(?,?,?,?,?)"

        self.cursor.execute(query,(labors.name,labors.surname,labors.wage,managers.departmentlabors.professions))

        self.connection.commit()
    def delete_labors(self,name):

        query = "Delete From labrar3 where name = ?"

        self.cursor.execute(query,(name,))

        self.connection.commit()
    def increase_pressure(self,name):

        query = "Select * From labrar3 where name = ?"

        self.cursor.execute(query,(name,))


        labrar3 = self.cursor.fetchall()

        if (len(labrar3) == 0):
            print("NO HERE...")
        else:
            pressure = labrar3[0][4]

            pressure += 1

            query2 = "Update labrar2 set pressure = ? where name = ?"

            self.cursor.execute(query2,(pressure,name))

            self.connection.commit()


### Definition of Classes
#### 1. Personnel Class

In [ ]:
class Personnel():
    def __init__(self,name,surname,pay_grade,zip_code,email,jobstart_date):
        print('Class of Field Personnel init function')
        self.name=name
        self.surname=surname
        self.jobstart_date=jobstart_date
        self.pay_grade=pay_grade
        self.email=email
        self.zip_code=zip_code
    def change_name(self,new_name):
        print('Name Changed')
        self.name=new_name
    def change_surname(self,new_surname):
        print('Surname Changed')
        self.surname=new_surname
    def change_email(self,new_email):
        print('Changing E-mail adress..')
        self.email=new_email
    def change_ZipCode(self,new_ZipCode):
        print('Zip Code changed')
        self.zip_code=new_ZipCode
    def change_jobstart_date(self,new_date):
        print('Date changed')
        self.jobstart_date=new_date
    def pay_grade_increase(self,accrual):
        print('Wage is increasing...')
        self.pay_grade+=accrual
    def pay_grade_decreasing(self,cutback):
        print('Wage is Decresing...')
        self.pay_grade-= cutback
    def show_informations(self):
        print('Personnel Informations Prepared...')
    def __str__(self):

        return "Name: {}\nSurname: {}\nPay_Grade: {}\nEmail: {}\nZip_Code: {}\nJobSStart_Date".format(self.name,self.surname,self.pay_grade,self.email,self.azip_code,self.jobstart_date)
